In [1]:
import requests
import pandas as pd
from typing import List, Dict
import time

In [2]:
# API call to ministry level expenses in federal budget
def fetch_budget_data(year: int) -> Dict:
    url = "https://bundeshaushalt.de/internalapi/budgetData"
    params = {
        'year': year,
        'account': 'expenses',
        'quota': 'target',
        'unit': 'single'
    }

    try:
        response = requests.get(url, params=params, timeout=10)

        if response.status_code == 200:
            data = response.json()
            return data
        else:
            print(f"     Error: Status Code {response.status_code}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"     Error fetching data: {e}")
        return None

def extract_children_data(data: Dict, year: int) -> List[Dict]:
    if not data or 'children' not in data:
        return []

    rows = []
    for child in data['children']:
        row = {
            'Jahr': year,
            'ID': child.get('id', ''),
            'Budgetnummer': child.get('budgetNumber', ''),
            'Kategorie': child.get('label', ''),
            'Wert_Euro': child.get('value', 0),
            'Relativer_Wert_Prozent': child.get('relativeValue', 0),
            'Relativer_Wert_zu_Eltern_Prozent': child.get('relativeToParentValue', 0)
        }
        rows.append(row)

    return rows

def main():
    years = range(2014, 2027)  # federal budget data from 2014 to 2026 since before buget IDs were different
    all_data = []

    # Fetch data for each year
    for year in years:
        print(f"\nYear {year}:")
        print(f"  → Fetching all ministries...")

        data = fetch_budget_data(year)

        if data:
            rows = extract_children_data(data, year)
            all_data.extend(rows)
            print(f"Found {len(rows)} ministries")
            print(f"Total budget: {sum(r['Wert_Euro'] for r in rows):,.2f} €")
        else:
            print(f"Could not fetch data for {year}")

        time.sleep(0.2)

    # Create DataFrame
    df = pd.DataFrame(all_data)

    # Check if we have data
    if df.empty:
        print("\nNo data retrieved. Please check if the API is accessible.")
        return

    # Sort by year and ID
    df = df.sort_values(['Jahr', 'ID'])

    # Export to CSV
    output_file = 'project_chart4_ministries_budget.csv'
    df.to_csv(output_file, index=False)

    print("\n" + "=" * 60)
    print(f"✓ Success! Data saved to '{output_file}'.")
    print(f"  Total of {len(df)} records exported.")

    # Show summary
    print(f"\nTotal budget per year:")
    print(df.groupby('Jahr')['Wert_Euro'].sum().apply(lambda x: f"{x:,.0f} €"))

    print(f"\nNumber of ministries per year:")
    print(df.groupby('Jahr')['Kategorie'].count())

if __name__ == "__main__":
    main()


Year 2014:
  → Fetching all ministries...
Found 22 ministries
Total budget: 296,500,000,000.00 €

Year 2015:
  → Fetching all ministries...
Found 22 ministries
Total budget: 306,900,000,000.00 €

Year 2016:
  → Fetching all ministries...
Found 23 ministries
Total budget: 316,900,000,000.00 €

Year 2017:
  → Fetching all ministries...
Found 23 ministries
Total budget: 329,100,000,000.00 €

Year 2018:
  → Fetching all ministries...
Found 23 ministries
Total budget: 343,600,000,000.00 €

Year 2019:
  → Fetching all ministries...
Found 23 ministries
Total budget: 356,400,000,000.00 €

Year 2020:
  → Fetching all ministries...
Found 23 ministries
Total budget: 508,529,758,000.00 €

Year 2021:
  → Fetching all ministries...
Found 24 ministries
Total budget: 572,725,714,000.00 €

Year 2022:
  → Fetching all ministries...
Found 25 ministries
Total budget: 495,791,475,000.00 €

Year 2023:
  → Fetching all ministries...
Found 25 ministries
Total budget: 461,211,782,000.00 €

Year 2024:
  → Fetc